# CSPB 3202 Final Project #

Tyler Kinkade, jaki9292@colorado.edu

GitHub: [https://github.com/jaki9292/rl-project](https://github.com/jaki9292/rl-project)

## Overview ##

This write-up reports on a small project to train and test a reinforcement learning algorithm (Russell & Norvig, 2022; Sutton & Barto, 2018) with the Gymnasium (2022) Python software package. 

This report is divided into the following sections: approach, results, discussion, and suggestions for future research.

## Approach ##

This section is divided into the following subsections: environment and game rules, models, methods and purpose, and problem solving procedure.

### Environment and Game Rules ###

Does it explain how the environment works and what the game rules are?


### Models ###

Does it explain clearly the model(s) of choices, the methods and purpose of tests and experiments?

Approximate reinforcement learning (Russell & Norvig, 2022; Sutton & Barto, 2018)



### Methods and Purpose ###

methods and purpose of tests and experiments





### Problem Solving Procedure ###

Does it show problem solving procedure- e.g. how the author solved and improved when an algorithm doesn't work well. Note that it's not about debugging or programming/implementation, but about when a correctly implemented algorithm wasn't enough for the problem and the author had to modify/add some features or techniques, or compare with another model, etc.

## Results ## 

show the result and interpretation of your experiment. Any iterative improvements summary.

demo clips

Does it include the results summary, interpretation of experiments and visualization (e.g. performance comparison table, graphs etc)?

## Discussion ## 

Does it include discussion (what went well or not and why), and suggestions for improvements or future work?

## Suggestions for Future Research ##




## References ##

Gymnasium. (2022). _Gymnasium documentation._ Farama Foundation. [https://gymnasium.farama.org/](https://gymnasium.farama.org/)

Russell, S., & Norvig, R. (2022). Artificial intelligence: A modern approach, (4th ed.). Pearson. 

Sutton, R. S., & Barto, A. G. (2018). Reinforcement learning: An introduction. MIT Press.